# VtemMax system

This is the forward model for a waveform given as the cchange of the rpiamry amgnetic field as a function of time in nT/s

In [1]:
import sys
import os
import numpy
import pyp223

## Forward problem 

### Model specification

In [2]:
# number of layers (inlcuding halfspace)
nlyr=2
# number of fiducials/stations
nstat=1
# layer restitvities for each fiducial (Ohm meters) do not include halfspace
res=numpy.array([300,1000])
# basement resistvity
pbres=1000.0
# layer thicknesses
thk=numpy.ones([(nlyr-1)*nstat])*25.0 
# number of thin plates
nplt=1
# plate location - easting
peast=numpy.array([50])
# plote location - northing
pnorth=numpy.array([25])
# plate location - depth
ptop=numpy.array([30])
# plate resistivity
pres=numpy.array([1.0])
# plate length 1
plngth1=numpy.array([100])
# plate length 2
plngth2=numpy.array([100])
# plate width 1
pwdth1=numpy.array([0.0])
# plate width 2
pwdth2=numpy.array([90])
# cell width
cellw = 25
# plate thickness
pthk=numpy.array([1])
# dip azimuth
pdzm=numpy.deg2rad(numpy.array([90]))
# dip
pdip=numpy.deg2rad(numpy.array([60]))
# plunge
plng=numpy.deg2rad(numpy.array([0]))

### Vtem system specification - waveform in nT/s

In [3]:
### Read gates and waveform
fh=open('LeroiAir.cfl')
lines=fh.readlines()
nsx=int(lines[2].split()[1])
nchnl=int(lines[2].split()[4])
swx=[]
waveform=[]

for i in range(nsx):
    fields=lines[3+i].split()
    swx.append(float(fields[0])/1000.0)
    waveform.append(fields[1])

topn=[]
tcls=[]
for i in range(nchnl):
    fields=lines[3+nsx+i].split()
    topn.append(float(fields[0])/1000.)
    tcls.append(float(fields[1])/1000.)

swx=numpy.array(swx)
waveform=numpy.array(waveform)
topn=numpy.array(topn)
tcls=numpy.array(tcls)

# number of components
ncmp=2
# active components
cmp=2
# number transmitter turns
ntrn=3
# transmitter area
txarea=531
# number of channels nchnl read from cfl
# time at which the channel opens topn read from cfl
# time at which the channel closes tcls read from cfl
# number of samples on waveform nsx read from cfl
# times for waveform swx read from cfl
# amplitude type AMPS 0
ampt=1
# amplitude  waveform read from cfl
# transimtter easting/x-pos
tx=numpy.array([225.0])
# transmitter northing/y-pos
ty=numpy.array([100.0])
# transimtter height
tz=numpy.array([105.0])
# transmitter azimuth
tazi=numpy.deg2rad(numpy.array([0.0]))
# transmitter inclination
tincl=numpy.deg2rad(numpy.array([6.]))
# receiver easting/x-pos
rx=numpy.array([225.])
# receiever northin/y-pos
ry=numpy.array([100.])
# receiver height/z-pos
rz=numpy.array([105.]) 
# transmiter receiver separation inline
trdx=numpy.array([0.])
# transmitter receiver separation crossline
trdy=numpy.array([0.])
# transmitter receiver separation vertical
trdz=numpy.array([0.])

### Model prediction

In [4]:
# response
xmodl=numpy.zeros([nchnl*ncmp])
# ijac - active elements of Jacobian
ijac=None
# jacobian
a=None
# lCounter for leroiair failures
leroiair_failure_count=0

In [5]:
leroiair=pyp223.LeroiAir()

In [6]:
prd=leroiair.formod_vtem_max_data(nlyr,nstat,res,pbres,thk,nplt,peast,pnorth,ptop,pres,
                                         plngth1,plngth2,pwdth1,pwdth2,pthk,cellw,
                                         pdzm,pdip,plng,
                                         ncmp,cmp,
                                          ntrn,txarea,
                                         nchnl,topn,tcls,
                                         nsx,swx,ampt,waveform,
                                         tx,ty,tz,tazi,tincl,
                                         rx,ry,rz,trdx,trdy,trdz,
                                         xmodl,
                                         leroiair_failure_count)
        

#### Difference in percentage between executable and wrapper

In [7]:
fh=open('LeroiAir.out')
lines=fh.readlines()
ref=numpy.zeros([nchnl,ncmp])

ref[:,0]=numpy.array(lines[612].split()[4:])
ref[:,1]=numpy.array(lines[622].split()[4:])
(prd-ref)/ref*100.0

array([[ 0.00221061,  0.00841476],
       [-0.00129015, -0.00156351],
       [-0.00024511, -0.01663698],
       [ 0.00030043,  0.01722498],
       [-0.00043597, -0.00388992],
       [-0.00032644, -0.00385923],
       [-0.00913755, -0.0232829 ],
       [ 0.01744496, -0.00092194],
       [-0.01238309,  0.00606527],
       [ 0.03038035,  0.00320638],
       [-0.00371807, -0.00179286],
       [ 0.00580893,  0.00355037],
       [-0.00963981, -0.01803164],
       [ 0.01313059,  0.02060348],
       [ 0.02313274,  0.04006326],
       [-0.01597157,  0.00461773],
       [-0.00358069, -0.00251774],
       [ 0.00152381, -0.00870713],
       [-0.00955952, -0.01219101],
       [-0.01460508, -0.00698462],
       [-0.01870645, -0.00801947],
       [-0.02925415, -0.0007933 ],
       [ 0.00252197,  0.00324536],
       [-0.00553192,  0.00913214],
       [-0.00273522, -0.00535064],
       [-0.00057926,  0.0376802 ]])